In [22]:
# Instructiuni:
# iau din train le dau la antrenat 
# dupa verific acuratetea pe validare
# dupa fac un predict pe cele de test si scriu intr-un csv imaginea si clasa lor si pe ala il incarc pe kaggle

In [23]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop

import tensorflow.compat as tf
#import tensorflow.compat.v1 as tf 
#tf.disable_v2_behavior()

from tensorflow import keras
from tensorflow.keras import layers

In [24]:
# citire imagini de antrenare plus etichete de antrenare
f = open ("train.txt")
train_images = []
train_labels = []
for line in enumerate(open("train.txt", "r")):
    v = line[1].split(',')
    image_name = 'train/'+v[0]
    label = int(v[1])
    train_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    train_images.append(image)
# transformare în nparray
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [25]:
# citire imagini de testare 
f = open ("test.txt")
test_images = []
for line in enumerate(open("test.txt", "r")):
    image_name = 'test/'+ line[1][:len(line[1])-1]
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    test_images.append(image)
# transformare în nparray
test_images = np.array(test_images)

In [26]:
# citire imagini de validare plus etichete de validare
f = open ("validation.txt")
validation_images = []
validation_labels = []
for line in enumerate(open("validation.txt", "r")):
    v = line[1].split(',')
    image_name = 'validation/'+v[0]
    label = int(v[1])
    validation_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    validation_images.append(image)
# transformare în nparray
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

In [27]:
nb_classes = 10 #10 categorii notate de la 0 la 9

x_train=train_images.astype('float32')
x_val=validation_images.astype('float32')

y_val=np_utils.to_categorical(validation_labels, nb_classes)
y_train=np_utils.to_categorical(train_labels, nb_classes)

In [28]:
# Marire esantion de date
# creez un esantion separat
# prin aplicarea de transformari aleatorii imaginilor de antrenament 
# Scop: evitarea overfitting-ului & expunerea modelului la mai multe cazuri de studiu  
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #rasucire orizontala
        layers.experimental.preprocessing.RandomRotation(0.1), #rotatie aleatorie a imaginii
    ]
)
# creare model de baza
base_model = keras.applications.Xception(
    weights="imagenet",  # incarcare date preantrenate din ImageNet
    input_shape=(150, 150, 3), #dimensiuni ImageNet
    include_top=False,
)  

# "Dezactivare" model de baza (Freezing) 
base_model.trainable = False

# Craere model nou 
inputs = keras.Input(shape=(150, 150, 3)) 
x = data_augmentation(inputs)  # Aplicare transformari aleatorii 

# Adaug un layer de normalizare pe intervalele [0,255] & [-1,1] => datele preantrenate au nevoie de normalizare
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# [-1,1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# "Batch normalization" -> model mai rapid si stabil, normalizare layer de intrare (prin recentrare si redimensionare)
# Modelul de baza contine layere batch-norm 
# Pastrare model in modul de inferenta (inference mode) -> pregatirea pentru fine-tuning  
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

# Pentru regularizare adaug un Dropout layer => Scop: reducere overfitting
x = keras.layers.Dropout(0.2)(x)  

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 150, 150, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 2049

In [29]:
model = Sequential()
model.add(Dense(100, input_shape =(x_train.shape[1],)))
model.add(Activation ('relu'))
model.add(Dense(200))
model.add(Activation ('sigmoid'))
model.add(Dense(nb_classes))
model.add(Activation ('softmax'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-6, decay=1e-6), metrics=['accuracy'])
history= model.fit (x_train, y_train, batch_size=200, epochs=1000, verbose=2,  validation_split=0.2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Epoch 1/1000
120/120 - 1s - loss: 0.2803 - accuracy: 0.3382 - val_loss: 0.2446 - val_accuracy: 0.4469
Epoch 2/1000
120/120 - 1s - loss: 0.2302 - accuracy: 0.4888 - val_loss: 0.2238 - val_accuracy: 0.4928
Epoch 3/1000
120/120 - 0s - loss: 0.2107 - accuracy: 0.5391 - val_loss: 0.2037 - val_accuracy: 0.5626
Epoch 4/1000
120/120 - 1s - loss: 0.1972 - accuracy: 0.5721 - val_loss: 0.1974 - val_accuracy: 0.5666
Epoch 5/1000
120/120 - 1s - loss: 0.1882 - accuracy: 0.5939 - val_loss: 0.1809 - val_accuracy: 0.6246
Epoch 6/1000
120/120 - 0s - loss: 0.1806 - accuracy: 0.6158 - val_loss: 0.1757 - val_accuracy: 0.6326
Epoch 7/1000
120/120 - 0s - loss: 0.1749 - accuracy: 0.6306 - val_loss: 0.1804 - val_accuracy: 0.6076
Epoch 8/1000
120/120 - 0s - loss: 0.1700 - accuracy: 0.6416 - val_loss: 0.1806 - val_accuracy: 0.6066
Epoch 9/1000
120/120 - 0s - loss: 0.1654 - accuracy: 0.6543 - val_loss: 0.1715 - val_accuracy: 0.6356
Epoch 10/1000
120/120 - 0s - loss: 0.1621 - accuracy: 0.6634 - val_loss: 0.1711 - 

Epoch 81/1000
120/120 - 0s - loss: 0.0701 - accuracy: 0.8740 - val_loss: 0.1529 - val_accuracy: 0.7182
Epoch 82/1000
120/120 - 0s - loss: 0.0707 - accuracy: 0.8714 - val_loss: 0.1527 - val_accuracy: 0.7157
Epoch 83/1000
120/120 - 0s - loss: 0.0689 - accuracy: 0.8780 - val_loss: 0.1567 - val_accuracy: 0.7090
Epoch 84/1000
120/120 - 0s - loss: 0.0688 - accuracy: 0.8772 - val_loss: 0.1576 - val_accuracy: 0.7104
Epoch 85/1000
120/120 - 0s - loss: 0.0677 - accuracy: 0.8804 - val_loss: 0.1611 - val_accuracy: 0.7105
Epoch 86/1000
120/120 - 0s - loss: 0.0669 - accuracy: 0.8827 - val_loss: 0.1527 - val_accuracy: 0.7137
Epoch 87/1000
120/120 - 0s - loss: 0.0659 - accuracy: 0.8839 - val_loss: 0.1589 - val_accuracy: 0.7107
Epoch 88/1000
120/120 - 0s - loss: 0.0658 - accuracy: 0.8840 - val_loss: 0.1551 - val_accuracy: 0.7162
Epoch 89/1000
120/120 - 1s - loss: 0.0652 - accuracy: 0.8845 - val_loss: 0.1533 - val_accuracy: 0.7137
Epoch 90/1000
120/120 - 0s - loss: 0.0650 - accuracy: 0.8844 - val_loss: 

Epoch 160/1000
120/120 - 0s - loss: 0.0307 - accuracy: 0.9581 - val_loss: 0.2046 - val_accuracy: 0.7104
Epoch 161/1000
120/120 - 0s - loss: 0.0304 - accuracy: 0.9600 - val_loss: 0.2063 - val_accuracy: 0.7144
Epoch 162/1000
120/120 - 0s - loss: 0.0308 - accuracy: 0.9590 - val_loss: 0.2071 - val_accuracy: 0.7090
Epoch 163/1000
120/120 - 0s - loss: 0.0296 - accuracy: 0.9621 - val_loss: 0.1943 - val_accuracy: 0.7245
Epoch 164/1000
120/120 - 0s - loss: 0.0298 - accuracy: 0.9598 - val_loss: 0.1985 - val_accuracy: 0.7199
Epoch 165/1000
120/120 - 0s - loss: 0.0296 - accuracy: 0.9615 - val_loss: 0.2055 - val_accuracy: 0.7137
Epoch 166/1000
120/120 - 0s - loss: 0.0288 - accuracy: 0.9625 - val_loss: 0.2107 - val_accuracy: 0.7094
Epoch 167/1000
120/120 - 1s - loss: 0.0285 - accuracy: 0.9629 - val_loss: 0.2109 - val_accuracy: 0.7105
Epoch 168/1000
120/120 - 0s - loss: 0.0286 - accuracy: 0.9623 - val_loss: 0.2142 - val_accuracy: 0.7115
Epoch 169/1000
120/120 - 0s - loss: 0.0276 - accuracy: 0.9660 - 

Epoch 239/1000
120/120 - 0s - loss: 0.0143 - accuracy: 0.9856 - val_loss: 0.2534 - val_accuracy: 0.7232
Epoch 240/1000
120/120 - 0s - loss: 0.0137 - accuracy: 0.9862 - val_loss: 0.2632 - val_accuracy: 0.7157
Epoch 241/1000
120/120 - 0s - loss: 0.0135 - accuracy: 0.9856 - val_loss: 0.2566 - val_accuracy: 0.7200
Epoch 242/1000
120/120 - 0s - loss: 0.0151 - accuracy: 0.9833 - val_loss: 0.2834 - val_accuracy: 0.6996
Epoch 243/1000
120/120 - 1s - loss: 0.0139 - accuracy: 0.9845 - val_loss: 0.2618 - val_accuracy: 0.7174
Epoch 244/1000
120/120 - 0s - loss: 0.0132 - accuracy: 0.9862 - val_loss: 0.2675 - val_accuracy: 0.7145
Epoch 245/1000
120/120 - 0s - loss: 0.0137 - accuracy: 0.9851 - val_loss: 0.2771 - val_accuracy: 0.7095
Epoch 246/1000
120/120 - 0s - loss: 0.0136 - accuracy: 0.9860 - val_loss: 0.2623 - val_accuracy: 0.7202
Epoch 247/1000
120/120 - 0s - loss: 0.0136 - accuracy: 0.9855 - val_loss: 0.2640 - val_accuracy: 0.7165
Epoch 248/1000
120/120 - 0s - loss: 0.0135 - accuracy: 0.9868 - 

Epoch 318/1000
120/120 - 1s - loss: 0.0095 - accuracy: 0.9898 - val_loss: 0.3124 - val_accuracy: 0.7140
Epoch 319/1000
120/120 - 1s - loss: 0.0086 - accuracy: 0.9914 - val_loss: 0.3146 - val_accuracy: 0.7160
Epoch 320/1000
120/120 - 1s - loss: 0.0109 - accuracy: 0.9873 - val_loss: 0.3351 - val_accuracy: 0.7042
Epoch 321/1000
120/120 - 1s - loss: 0.0083 - accuracy: 0.9917 - val_loss: 0.3115 - val_accuracy: 0.7212
Epoch 322/1000
120/120 - 1s - loss: 0.0079 - accuracy: 0.9917 - val_loss: 0.3248 - val_accuracy: 0.7147
Epoch 323/1000
120/120 - 1s - loss: 0.0097 - accuracy: 0.9902 - val_loss: 0.3360 - val_accuracy: 0.6986
Epoch 324/1000
120/120 - 1s - loss: 0.0082 - accuracy: 0.9915 - val_loss: 0.3156 - val_accuracy: 0.7180
Epoch 325/1000
120/120 - 1s - loss: 0.0091 - accuracy: 0.9901 - val_loss: 0.3133 - val_accuracy: 0.7172
Epoch 326/1000
120/120 - 1s - loss: 0.0087 - accuracy: 0.9912 - val_loss: 0.3213 - val_accuracy: 0.7115
Epoch 327/1000
120/120 - 1s - loss: 0.0084 - accuracy: 0.9907 - 

...

Epoch 634/1000
120/120 - 1s - loss: 0.0059 - accuracy: 0.9947 - val_loss: 0.4476 - val_accuracy: 0.7199
Epoch 635/1000
120/120 - 1s - loss: 0.0057 - accuracy: 0.9943 - val_loss: 0.4485 - val_accuracy: 0.7195
Epoch 636/1000
120/120 - 1s - loss: 0.0063 - accuracy: 0.9943 - val_loss: 0.4463 - val_accuracy: 0.7162
Epoch 637/1000
120/120 - 1s - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.4547 - val_accuracy: 0.7217
Epoch 638/1000
120/120 - 1s - loss: 0.0047 - accuracy: 0.9950 - val_loss: 0.4486 - val_accuracy: 0.7195
Epoch 639/1000
120/120 - 1s - loss: 0.0064 - accuracy: 0.9937 - val_loss: 0.4442 - val_accuracy: 0.7220
Epoch 640/1000
120/120 - 1s - loss: 0.0048 - accuracy: 0.9958 - val_loss: 0.4449 - val_accuracy: 0.7185
Epoch 641/1000
120/120 - 1s - loss: 0.0041 - accuracy: 0.9967 - val_loss: 0.4598 - val_accuracy: 0.7137
Epoch 642/1000
120/120 - 1s - loss: 0.0052 - accuracy: 0.9952 - val_loss: 0.4525 - val_accuracy: 0.7169
Epoch 643/1000
120/120 - 1s - loss: 0.0024 - accuracy: 0.9978 - 

Epoch 713/1000
120/120 - 1s - loss: 0.0031 - accuracy: 0.9977 - val_loss: 0.4688 - val_accuracy: 0.7202
Epoch 714/1000
120/120 - 1s - loss: 0.0045 - accuracy: 0.9959 - val_loss: 0.4652 - val_accuracy: 0.7239
Epoch 715/1000
120/120 - 1s - loss: 0.0031 - accuracy: 0.9978 - val_loss: 0.4746 - val_accuracy: 0.7225
Epoch 716/1000
120/120 - 1s - loss: 0.0052 - accuracy: 0.9957 - val_loss: 0.4723 - val_accuracy: 0.7205
Epoch 717/1000
120/120 - 1s - loss: 0.0018 - accuracy: 0.9983 - val_loss: 0.4759 - val_accuracy: 0.7227
Epoch 718/1000
120/120 - 1s - loss: 0.0046 - accuracy: 0.9954 - val_loss: 0.4738 - val_accuracy: 0.7184
Epoch 719/1000
120/120 - 1s - loss: 0.0023 - accuracy: 0.9978 - val_loss: 0.4758 - val_accuracy: 0.7200
Epoch 720/1000
120/120 - 1s - loss: 0.0053 - accuracy: 0.9955 - val_loss: 0.4706 - val_accuracy: 0.7229
Epoch 721/1000
120/120 - 1s - loss: 0.0029 - accuracy: 0.9977 - val_loss: 0.4768 - val_accuracy: 0.7202
Epoch 722/1000
120/120 - 1s - loss: 0.0058 - accuracy: 0.9947 - 

Epoch 792/1000
120/120 - 1s - loss: 0.0042 - accuracy: 0.9962 - val_loss: 0.4963 - val_accuracy: 0.7200
Epoch 793/1000
120/120 - 1s - loss: 0.0041 - accuracy: 0.9967 - val_loss: 0.4945 - val_accuracy: 0.7224
Epoch 794/1000
120/120 - 1s - loss: 0.0051 - accuracy: 0.9955 - val_loss: 0.4894 - val_accuracy: 0.7179
Epoch 795/1000
120/120 - 1s - loss: 0.0025 - accuracy: 0.9982 - val_loss: 0.5010 - val_accuracy: 0.7210
Epoch 796/1000
120/120 - 1s - loss: 0.0069 - accuracy: 0.9940 - val_loss: 0.5092 - val_accuracy: 0.7079
Epoch 797/1000
120/120 - 1s - loss: 0.0032 - accuracy: 0.9970 - val_loss: 0.4909 - val_accuracy: 0.7210
Epoch 798/1000
120/120 - 1s - loss: 0.0048 - accuracy: 0.9958 - val_loss: 0.4977 - val_accuracy: 0.7169
Epoch 799/1000
120/120 - 1s - loss: 0.0048 - accuracy: 0.9960 - val_loss: 0.4907 - val_accuracy: 0.7232
Epoch 800/1000
120/120 - 1s - loss: 0.0048 - accuracy: 0.9968 - val_loss: 0.4945 - val_accuracy: 0.7200
Epoch 801/1000
120/120 - 1s - loss: 0.0045 - accuracy: 0.9960 - 

Epoch 871/1000
120/120 - 1s - loss: 0.0029 - accuracy: 0.9975 - val_loss: 0.5084 - val_accuracy: 0.7215
Epoch 872/1000
120/120 - 1s - loss: 0.0043 - accuracy: 0.9964 - val_loss: 0.5072 - val_accuracy: 0.7217
Epoch 873/1000
120/120 - 1s - loss: 0.0034 - accuracy: 0.9968 - val_loss: 0.5085 - val_accuracy: 0.7222
Epoch 874/1000
120/120 - 1s - loss: 0.0044 - accuracy: 0.9965 - val_loss: 0.5095 - val_accuracy: 0.7190
Epoch 875/1000
120/120 - 1s - loss: 0.0041 - accuracy: 0.9960 - val_loss: 0.5120 - val_accuracy: 0.7199
Epoch 876/1000
120/120 - 1s - loss: 0.0039 - accuracy: 0.9964 - val_loss: 0.5821 - val_accuracy: 0.6914
Epoch 877/1000
120/120 - 1s - loss: 0.0027 - accuracy: 0.9975 - val_loss: 0.5110 - val_accuracy: 0.7207
Epoch 878/1000
120/120 - 1s - loss: 0.0050 - accuracy: 0.9959 - val_loss: 0.5108 - val_accuracy: 0.7190
Epoch 879/1000
120/120 - 1s - loss: 0.0046 - accuracy: 0.9960 - val_loss: 0.5349 - val_accuracy: 0.7044
Epoch 880/1000
120/120 - 1s - loss: 0.0021 - accuracy: 0.9980 - 

Epoch 950/1000
120/120 - 1s - loss: 0.0051 - accuracy: 0.9960 - val_loss: 0.5263 - val_accuracy: 0.7169
Epoch 951/1000
120/120 - 1s - loss: 0.0044 - accuracy: 0.9965 - val_loss: 0.5297 - val_accuracy: 0.7164
Epoch 952/1000
120/120 - 1s - loss: 0.0024 - accuracy: 0.9984 - val_loss: 0.5232 - val_accuracy: 0.7204
Epoch 953/1000
120/120 - 1s - loss: 0.0027 - accuracy: 0.9976 - val_loss: 0.5236 - val_accuracy: 0.7234
Epoch 954/1000
120/120 - 1s - loss: 0.0028 - accuracy: 0.9978 - val_loss: 0.5261 - val_accuracy: 0.7234
Epoch 955/1000
120/120 - 1s - loss: 0.0052 - accuracy: 0.9959 - val_loss: 0.6058 - val_accuracy: 0.6919
Epoch 956/1000
120/120 - 1s - loss: 0.0030 - accuracy: 0.9975 - val_loss: 0.5229 - val_accuracy: 0.7232
Epoch 957/1000
120/120 - 1s - loss: 0.0040 - accuracy: 0.9966 - val_loss: 0.5244 - val_accuracy: 0.7195
Epoch 958/1000
120/120 - 1s - loss: 0.0030 - accuracy: 0.9973 - val_loss: 0.5253 - val_accuracy: 0.7222
Epoch 959/1000
120/120 - 1s - loss: 0.0057 - accuracy: 0.9958 - 

In [30]:
# Antrenare model principal
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


In [31]:
# "Fine tunning"
# "Activare" (Unfreeze) model de baza aflat in "inference mode"

base_model.trainable = True
model.summary()


model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # rata de invatare mica
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history= model.fit (x_train, y_train, batch_size=200, epochs=20, verbose=2,  validation_split=0.2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               102500    
_________________________________________________________________
activation_9 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_10 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2010      
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
Total params: 124,710
Trainable params: 124,710
Non-trainable params: 0
________________________________________________

In [32]:
#Neural Network Classifier 

# nb_classes = 10 #10 categorii notate de la 0 la 9

# x_train=train_images.astype('float32')
# x_val=validation_images.astype('float32')

# y_val=np_utils.to_categorical(validation_labels, nb_classes)
# y_train=np_utils.to_categorical(train_labels, nb_classes)

# model = Sequential()
# model.add(Dense(100, input_shape =(x_train.shape[1],)))
# model.add(Activation ('relu'))
# model.add(Dense(200))
# model.add(Activation ('sigmoid'))
# model.add(Dense(nb_classes))
# model.add(Activation ('softmax'))

# model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-6, decay=1e-6), metrics=['accuracy'])
# history= model.fit (x_train, y_train, batch_size=200, epochs=1000, verbose=2,  validation_split=0.2)
# loss_and_metrics = model.evaluate(x_val, y_val)
# print(loss_and_metrics)
